In [ ]:
import pandas as pd

In [ ]:
import re

In [ ]:
import numpy as np

In [ ]:
timesData = pd.read_csv("dataframe/timesData.csv")
timesData.head()

In [ ]:
timesData.info()

In [ ]:
timesData['income']=pd.to_numeric(timesData['income'], errors='coerce')

In [ ]:
shanghaiData = pd.read_csv("dataframe/shanghaiData.csv")
shanghaiData.head()

In [ ]:
school_and_country = pd.read_csv("dataframe/school_and_country_table.csv")
school_and_country.head()

In [ ]:
cwurData = pd.read_csv("dataframe/cwurData.csv")
cwurData.head()

In [ ]:
educational_attainment = pd.read_csv("dataframe/educational_attainment_supplementary_data.csv")
educational_attainment.head()

In [ ]:
education_expenditure = pd.read_csv("dataframe/education_expenditure_supplementary_data.csv",engine='python')
education_expenditure.head()

1. For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframe

In [ ]:
most_recent_data=timesData.loc[(timesData.groupby('university_name')['year'].idxmax())]
most_recent_data.head()

In [ ]:
least_recent_data=timesData.loc[(timesData.groupby('university_name')['year'].idxmin())]
least_recent_data.head()

2. For each university, compute the improvement in "income" between the least recent and the most recent data points

In [ ]:
punto2=pd.merge(most_recent_data, least_recent_data, on='university_name')
punto2.head()

In [ ]:
punto2.columns

In [ ]:
def calcola_perc(riga):
    return riga['votes_first_vote']/ risultato[riga['state']] *100

In [ ]:
punto2['income_improvement']=punto2['income_y']-punto2['income_x']
punto2.head()

3. Find the university with the largest increase computed in the previous point

In [ ]:
punto2.loc[punto2['income_improvement']>= punto2['income_improvement'].max()]['university_name'] #possiamo anche togliere l'ultima quadra


4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range

In [ ]:
most_recent_s=shanghaiData.loc[(shanghaiData.groupby('university_name')['year'].idxmax())]
most_recent_c=cwurData.loc[(cwurData.groupby('institution')['year'].idxmax())]
#most_recent_data

In [ ]:
most_recent_data['world_rank']

In [ ]:
lista = most_recent_data['world_rank']
df =pd.DataFrame(lista)
df.head()

In [ ]:
df['min_range'] = df['world_rank'].str.extract(r'^\=?(\d+)')
df['max_range'] = df['world_rank'].str.extract(r'(\d+$)')
df.head()

In [ ]:
df['min_range']=pd.to_numeric(df['min_range'], errors='coerce')
df['max_range']=pd.to_numeric(df['max_range'], errors='coerce')
df.info()

In [ ]:
most_recent_data['world_rank_t'] = (df['max_range'] + df['min_range'])//2
most_recent_data['world_rank_t'].head()

In [ ]:
#rifacciamo per s
lista_s = most_recent_s['world_rank']
df_s =pd.DataFrame(lista_s)
df_s['min_range'] = df_s['world_rank'].str.extract(r'^\=?(\d+)')
df_s['max_range'] = df_s['world_rank'].str.extract(r'(\d+$)')
df_s['min_range']=pd.to_numeric(df_s['min_range'], errors='coerce')
df_s['max_range']=pd.to_numeric(df_s['max_range'], errors='coerce')
most_recent_s['world_rank_s'] = (df_s['max_range'] + df_s['min_range'])//2
most_recent_s.head()

In [ ]:
most_recent_c.info()

In [ ]:
most_recent = pd.merge(most_recent_data, most_recent_s, on='university_name', how='left' )
most_recent

In [ ]:
most_recent1 = pd.merge(most_recent, most_recent_c, left_on= 'university_name', right_on = 'institution', how='left')
most_recent1

In [ ]:
most_recent1.columns

In [ ]:
most_recent1['HighScore'] = most_recent1[['world_rank_t','world_rank','world_rank_s']].max(axis = 1)
most_recent1['LowScore'] = most_recent1[['world_rank_t','world_rank','world_rank_s']].min(axis = 1)
most_recent1['maximum_difference'] = most_recent1['HighScore']-most_recent1['LowScore']
most_recent1[['university_name','maximum_difference','HighScore','LowScore','world_rank_t','world_rank','world_rank_s']]

5. Consider only the most recent data point of the times dataset. Compute the number of male and female students for each country.

In [ ]:
most_recent_data.info()

In [ ]:
most_recent_data['female'] = most_recent_data['female_male_ratio'].str.extract(r'(^\d+)')
most_recent_data['male'] = most_recent_data['female_male_ratio'].str.extract(r'(\d+$)')
most_recent_data.head()

In [ ]:
most_recent_data['male']=pd.to_numeric(most_recent_data['male'], errors='coerce')
most_recent_data['female']=pd.to_numeric(most_recent_data['female'], errors='coerce')

In [ ]:
most_recent_data.info()

In [ ]:
most_recent_data['num_students'] = most_recent_data['num_students'].str.replace(',','')
most_recent_data.head()

In [ ]:
most_recent_data['num_students']=pd.to_numeric(most_recent_data['num_students'], errors='coerce')
most_recent_data.head()

In [ ]:
most_recent_data['num_male']= most_recent_data['male'] * most_recent_data['num_students'] // 100
most_recent_data.head()

In [ ]:
most_recent_data['num_fem']= most_recent_data['female'] * most_recent_data['num_students'] // 100
most_recent_data.head()

6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities)

In [ ]:
most_recent_data['rapporto'] = most_recent_data['num_fem'] / most_recent_data['num_male']
media = most_recent_data['rapporto'].notnull().mean()

In [ ]:
punto6 = most_recent_data.loc[most_recent_data['rapporto'] <= media]
punto6.head()

In [ ]:
punto6['university_name']

7. For each country, compute the fraction of the students in the country that are in one of the universities computed in the previous point (that is, the denominator of the ratio is the total number of students over all universities in the country).

In [ ]:
#denominatore
deno = most_recent_data.groupby('country', as_index = True)['num_students'].sum()
#numeratore
deno

In [ ]:
num=punto6.groupby('country')['num_students'].sum()

In [ ]:
num.dtype

In [ ]:
deno.dtype

In [ ]:
fraction=num/deno
fraction

In [ ]:
#most_recent_data.loc[most_recent_data.groupby('country').sum()['num_students']]

8. Read the file educational_attainment_supplementary_data.csv, discarding any row with missing country_name or series_name

In [ ]:
educational_attainment = pd.read_csv("dataframe/educational_attainment_supplementary_data.csv")
educational_attainment.head()

In [ ]:
educ = educational_attainment[(educational_attainment['country_name'].notnull()) | (educational_attainment['series_name'].notnull())]
educ.head()

9. From attainment build a dataframe with the same data, but with 4 columns: country_name, series_name, year, value

In [ ]:
ed_pulito= educ.dropna(axis='columns', how='all')
ed_pulito.head()

In [ ]:
pd.melt?

In [ ]:
ed_pulito.columns[2:]

In [ ]:
pd.melt(ed_pulito, id_vars=['country_name','series_name'], value_vars=['1985', '1986', '1987', '1990', '1991', '1992', '1993', '1995', '1996',
       '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007', '2008', '2009', '2010', '2011', '2012', '2013'])


10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).

In [171]:
tabella_merge=pd.merge(timesData,cwurData,how='left', left_on='university_name', right_on='institution')
tabella_merge=pd.merge(tabella_merge,shanghaiData,how='left', on='university_name')
tabella_merge=tabella_merge.rename(columns={'world_rank':'times','world_rank_x':'cwur','world_rank_y':'shanghai' })
tabella_merge=tabella_merge[['university_name','times','cwur','shanghai']]
tabella_merge['shanghai']=tabella_merge['shanghai'].replace(np.nan, 'Dato mancante')
tabella_merge['times']=tabella_merge['times'].replace(np.nan, 'Dato mancante')
tabella_merge['cwur']=tabella_merge['cwur'].replace(np.nan, 'Dato mancante')

In [172]:
def counter(col1,col2,col3):
    count=0
    if (col1!='Dato mancante'):
        count+=1
    if (col2!='Dato mancante'):
        count+=1
    if (col3!='Dato mancante'):
        count+=1

    return count


In [173]:
tabella_merge['counter']=tabella_merge.apply(lambda x:counter(x.times,x.cwur,x.shanghai), axis=1)
tabella_merge.tail(60)

,university_name,times,cwur,shanghai,counter
35511,Texas Tech University,301-400,601-800,Dato mancante,2
35512,Texas Tech University,305-402,601-800,Dato mancante,2
35513,Texas Tech University,303-401,601-800,Dato mancante,2
35514,Texas Tech University,303-401,601-800,Dato mancante,2
35515,Texas Tech University,301-400,601-800,Dato mancante,2
35516,Texas Tech University,401-500,601-800,Dato mancante,2
35517,Texas Tech University,401-500,601-800,Dato mancante,2
35518,Tokai University,Dato mancante,601-800,589,2
35519,Tokai University,Dato mancante,601-800,622,2
35520,Tokushima University,Dato mancante,601-800,Dato mancante,1


11. In the times ranking, compute the number of times each university appears

In [ ]:
punto11 = timesData.groupby('university_name', as_index = False)['world_rank'].count()
punto11

12. Find the universities that appear at most twice in the times ranking.

In [ ]:
punto12 = punto11[punto11['world_rank']<= 2]
punto12

13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).

In [78]:
punto13 = pd.merge(timesData, cwurData, left_on=['university_name','year'], right_on = ['institution','year'])
punto13= pd.merge(punto13, shanghaiData, on= ['university_name', 'year'])
punto13= punto13.rename(columns= {'world_rank_x':'times','world_rank_y': 'cwur','world_rank' : 'shanghai'})


In [103]:
punto13= punto13[['university_name','year','times','cwur','shanghai']]
punto13.head()

In [99]:
punto13.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 551 entries, 0 to 550
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   university_name  551 non-null    object
 1   year             551 non-null    int64 
 2   times            551 non-null    object
 3   cwur             551 non-null    int64 
 4   shanghai         551 non-null    object
dtypes: int64(2), object(3)
memory usage: 25.8+ KB


In [102]:
punto13 = punto13.loc[(punto13['times'] == punto13['shanghai'])]
punto13


,university_name,year,times,cwur,shanghai
2,Stanford University,2012,2,3,2
6,University of Chicago,2012,9,11,9
8,Yale University,2012,11,8,11
60,Stanford University,2013,2,2,2
67,Yale University,2013,11,10,11
70,University of Pennsylvania,2013,15,12,15
79,"University of California, Santa Barbara",2013,35,37,35
109,Princeton University,2014,6,9,6
111,University of Chicago,2014,9,8,9
112,Yale University,2014,11,10,11


In [105]:
punto13['times']=pd.to_numeric(punto13['times'], errors='coerce')
punto13['shanghai']=pd.to_numeric(punto13['shanghai'], errors='coerce')
punto13.loc[(punto13['times'] == punto13['cwur'])]

,university_name,year,times,cwur,shanghai
60,Stanford University,2013,2,2,2
